In [ ]:
%matplotlib inline
#%matplotlib notebook
from datetime import datetime
import numpy as np
import os
import matplotlib
#matplotlib.rcParams['savefig.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
matplotlib.rcParams["font.size"] = 20
import matplotlib.pyplot as plt

In [ ]:
def plot_logfile(fname):
    startmarks = []
    results = []
    with open(fname, 'r') as f:
        start = False
        for line in f.readlines():
            if 'FLASH log file:' in line:
                start = True
            if 'Number of MPI tasks:' in line:
                nProc = int(line.split()[-1])
            if '[GRID amr_refine_derefine] min blks' in line:
                tot_blks = int(line.split()[-1])
                max_blks = int(line.split()[-4])
            if '[Particles_getGlobalNum]' in line:
                nPart = line.split()[-1]
            if 'step: n=' in line:
                #print line
                dummy, date, time, dummy, dummy, step, t, dt = line.split()
                dtime = datetime.strptime(date+' '+time, '%m-%d-%Y %H:%M:%S.%f')
                step = int(step.lstrip('n='))
                t = float(t.lstrip('t='))
                dt = float(dt.lstrip('dt='))
                results.append([dtime, step, t, dt, max_blks, tot_blks, nPart])
                if start:
                    startmarks.append([step])
                    start = False
                #result = dict((name,eval(name)) for name in ['dtime','step','t','dt'] )
            
    dtimes, steps, ts, dts, max_blkss, tot_blkss, nParts = zip(*results)
    #startsteps = zip(*startmarks)
    dtimes = list(dtimes)
    dt_steps = [(dtimes[i+1]-dtimes[i]).total_seconds() for i in range(len(dtimes)-1)]
    plt.clf()
    plt.scatter(steps[1:], dt_steps, s=1, linewidth=0, label='t/step')
    plt.scatter(steps[1:], 10*np.array(dt_steps)/np.array(max_blkss[1:]), s=1, c='g', linewidth=0, label='t/10block/step')
    #print np.mean((np.array(dt_steps)/np.array(max_blkss[1:]))*100)
    plt.ylim(0.2, 1000)
    plt.xlabel('# of step')
    plt.semilogy()
    plt.ylabel('t (sec)')
    plt.legend(loc=2)
    
    # Plot start steps
    plt.vlines(startmarks,0.2,1000, color='grey', linestyles='--', alpha=0.5,)
    
    plt.twinx()
    plt.scatter(steps[1:], max_blkss[1:], s=1, linewidth=0, color='r', label='max blks')
    plt.ylabel('max blocks / processor')
    #plt.scatter(steps[1:], [ntot/320 for ntot in tot_blkss[1:]], s=1, linewidth=0, color='orange', label='tot blks')
    plt.ylim(0, 600)
    #plt.twinx()
    #plt.scatter(steps[1:], dts[1:], s=1, linewidth=0, color='g', label='dt')
    plt.xlim(0,steps[-1]+100)
    plt.legend(loc=1)
    plt.title(fname.split('/')[-2])
    plt.show()

In [ ]:
#%pdb
fnames = ['/home/ychen/data/0only_1106_M3_h1/MHD_Jet.log',
          '/home/ychen/data/0only_1204_M24_b01/MHD_Jet.log',
          '/home/ychen/data/0only_0529_h1/MHD_Jet.log']

for fname in fnames:
    plot_logfile(fname)


In [ ]:
def get_nozzledata(dirname):
    try:
        nozzledata = np.genfromtxt(os.path.join(dirname, 'nozzleVec.dat'))
        
        ttnoz = nozzledata[:,1]
        xx = nozzledata[:,2]
        yy = nozzledata[:,3]
        zz = nozzledata[:,4]
        thetas = np.arccos(zz)/np.pi*180
        phis = np.arctan2(yy, xx)
        return ttnoz, xx, yy, zz, thetas, phis
    except:
        pass

def get_dat(dirname):
    try:
        data = np.genfromtxt(os.path.join(dirname, 'MHD_Jet.dat'))
    except:
        data = np.genfromtxt(os.path.join(dirname, 'MHD_Jet_10Myr.dat'))
    tt = data[:,0]
    Masses = data[:,1]
    Etots = data[:,5]
    Ekins = data[:,6]
    Eints = data[:,7] 
    Emags = data[:,8]
    Ptots = (Etots[1:]-Etots[:-1])/(tt[1:]-tt[:-1])
    Pkins = (Ekins[1:]-Ekins[:-1])/(tt[1:]-tt[:-1])
    Pints = (Eints[1:]-Eints[:-1])/(tt[1:]-tt[:-1])
    #print max(Ptots), max(Pkins), max(Pints)
    return tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints

In [ ]:
#dirname = '/home/ychen/data/0529_L45_M10_b1_h1'
#dirname = '/home/ychen/d9/FLASH4/stampede/0916_L45_M10_b1_hinf_10Myr/'
#dirnames = ['/home/ychen/data/0529_L45_M10_b1_h1',\
#            '/home/ychen/data/0605_L45_M10_b1_h0',\
#            '/home/ychen/data/0605_L45_M10_b1_hinf_all',
#            '/home/ychen/data/0602_L45_M10_hydro_all']
dirnames = ['/home/ychen/data/0only_0529_h1',\
            '/home/ychen/data/0only_1106_M3_h1',\
            '/home/ychen/data/0only_1110_h0_rerun',\
            '/home/ychen/data/0only_1111_M10_b01',\
            '/home/ychen/data/0only_1204_M24_b01'
            ]
table = []
for dirname in dirnames:
    table.append(get_dat(dirname))

In [ ]:
names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['yellow', 'r', 'b', 'g', 'cyan']
plt.figure(figsize=(10,8))
for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.plot(tt[1:]/3.154E13, (tt[1:]-tt[:-1]), c=colors[i], label=names[i], lw=0, marker='.', markersize=1, alpha=0.7)


#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+9E59)
#plt.ylim(0, 4E58)
plt.ylim(0,9E9)
plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('dt')
plt.xlabel('t (Myr)')
plt.legend(loc=2)
plt.show()

In [ ]:
## names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']
for i in range(len(table)):
    print i
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.plot(tt[:]/3.154E13, (Eints[:]-Eints[0])/Emags[:], c=colors[i], label=names[i].split('_')[-1])
    plt.plot(tt[:]/3.154E13, (Ekins[:]-Ekins[0])/Emags[:], ':', c=colors[i], label='Ekin/Emag')

plt.legend(loc=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(2.563E62, 2.563E62+1E60)
#plt.ylim(0, 4E57)
#plt.ylim(0,400)
#plt.xlim(0, 25)
#plt.semilogy()
#plt.ylabel('Emag (erg)')
plt.ylabel(u'$\Delta$E/Emag')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']
for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.scatter(tt[:]/3.154E13, Ekins[:], s=1, c=colors[i], linewidth=0, label=names[i])

plt.legend(loc=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
plt.ylim(0.0, 2E59)
#plt.ylim(0, 4E57)
#plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('Ekin (erg)')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']
for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.scatter(tt[:]/3.154E13, Eints[:], s=1, c=colors[i], linewidth=0, label=names[i])

plt.legend(loc=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
plt.ylim(2.563E62, 2.563E62+1E60)
#plt.ylim(0, 4E57)
#plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('Eint (erg)')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']
for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.scatter(tt[:]/3.154E13, Masses[:], s=1, c=colors[i], linewidth=0, label=names[i])

plt.legend(loc=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(1.81313E46, 1.81313E46+2E41)
#plt.ylim(0, 4E57)
#plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel('Mass (g)')
plt.xlabel('t (Myr)')
plt.show()

In [ ]:
def Etot_lin(tt, E0=2.5637E62):
    return E0 + tt*1E45

names = [dirname.split('/')[-1] for dirname in dirnames]
colors = ['r', 'b', 'g', 'cyan']

for i in range(len(table)):
    tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
    plt.plot(tt[:]/3.154E13, Etots-Etot_lin(tt,E0=0), '-', c=colors[i])
    plt.plot(tt[:]/3.154E13, Etots-Emags-Etot_lin(tt,E0=0), ':', c=colors[i], label=names[i])
    

plt.legend(loc=2)
#plt.scatter(tt[:], Etots[:], s=1, linewidth=0, label='dt')
#plt.ylim(-2E58, 10E58)
#plt.ylim(0, 4E57)
#plt.xlim(0, 25)
#plt.semilogy()
plt.ylabel(u'$\Delta$ Etot (erg)')
plt.xlabel('t (Myr)')
plt.show()


In [ ]:
i=0
tt, Masses, Etots, Ekins, Eints, Emags, Ptots, Pkins, Pints = table[i]
dirname = dirnames[i]
fig = plt.figure(figsize=(8,6))
plt.scatter(tt[1:]/3.15569E13, (Etots[1:]-Etots[:-1])/(tt[1:]-tt[:-1]), s=1, c='b', linewidth=0, label='Total Power')
plt.scatter(tt[1:]/3.15569E13, (Ekins[1:]-Ekins[:-1])/(tt[1:]-tt[:-1]), s=1, c='r', linewidth=0, label='Kinetic Power')
plt.scatter(tt[1:]/3.15569E13, (Eints[1:]-Eints[:-1])/(tt[1:]-tt[:-1]), s=1, c='g', linewidth=0, label='Internal Power')
plt.scatter(tt[1:]/3.15569E13, (Emags[1:]-Emags[:-1])/(tt[1:]-tt[:-1]), s=1, c='purple', linewidth=0, label='Magnetic Power')
#plt.ylim(1E44, 1.5E45)
plt.ylim(-0E45, 2E45)
plt.ylabel('power (erg/s)')
plt.axhline(1E45, lw=0.5, c='grey')
plt.axhline(1.1E45, lw=0.5, c='grey')
#plt.ylim(1E44, 5E45)
#plt.ylim(1E38, 4E44)
plt.legend(loc=2)
#plt.semilogy()
#plt.twinx()

#plt.scatter(ttnoz/3.15569E13, thetas, s=1, linewidth=0, c='y', alpha=0.3, label='theta')
#plt.scatter(ttnoz, phis, s=1, linewidth=0, c='g', alpha=0.5, label='phi')
plt.legend(loc=2)
plt.xlim(0, 20)
plt.xlabel('t (Myr)')
plt.title(dirname.split('/')[-1])
plt.show()

In [ ]:
# plt.scatter(tt[1:], (Masses[1:]-Masses[:-1])/(tt[1:]-tt[:-1]), s=1, linewidth=0, label='Mass Flux')
plt.ylabel('mass flux (g/s)')
plt.xlabel('t (s)')
#plt.semilogy()